# Final project exercises

In the previous notebook we introduced questING, our final project for this course. In this notebook you will be asked to complete or implement some tests. Note that implementing some of the tests might require changes in the code, but we'll try to tell you upfront if code changes are expected.

All the tests and code you might need to change are located in the ```final-project/tests``` and ```final-project/questing``` folders respectively. You can find the solutions inside the ```final-project-solutions```, following the same structure.

## Goal

The goal is not that your solution is exactly the same as the proposed solution. As it usually happens with any development, there are many ways to achieve the same thing, and although some might be more correct than others, it often comes down to a matter of taste too. So just make sure you understand the proposed solution and compare whether you are solving the same problem, and not only what option you prefer, but what would be more maintainable and readable. In this regard, the simplest solution is often the best.

## Running the tests

The tests are meant to be executed often, so feel free to run them as often as you want. If you're inside the ```final-project``` directory, you can run the test with coverage with this command:

```bash
pytest --cov-report term-missing --cov=questing tests
```

### Running a single test file

In some cases, you might want to run just one of the test files, instead all of them. This is often the case for big projects, where the tests can start to take quite some time and you don't want to wait everytime you test. You can just provide the name of the file you want to test, instead of a folder:

```bash
pytest --cov-report term-missing --cov=questing tests/hero.py
```

### Running a single method from a file

Finally, it could also be the case that you only want to run one of the methods you have in your file, not all of them. You can do that too, by providing the file name AND the method name, separated by a double colon:

```bash
pytest --cov-report term-missing --cov=questing tests/hero.py::test_hero_name
```

## Exercise - Add tests for the stats of the different heroes.

In this first exercise, you need to add tests to make sure that the different stats of the heroes are correct. More precisely, you will have to add tests to check the following:

- A Warrior has Power 4, Armor 1, Speed 1 and Health 10.
- A Rogue has Power 3, Attack Range 3, Speed 2 and Health 9
- A Mage has Power 3, Attack Range 2, Armor 2, Speed 1 and Health 8

Once you're done, you can compare your solution with the one proposed. How many tests did you add? Would it be better to have more (or less tests)?

In the proposed solution, we have implemented one test per class, meaning that we have multiple assertions on every test. While this might be OK, in general the best practice is to try to have only one assertion per test, unless asserting one thing doesn't make sense without the other. So if you have create one test per class and attribute, your solution is probably better than the one provided.

The only benefit of having a single test per class is readability, since someone reading the test can see all the expected attributes very easily in one go, instead of having to read multiple tests.

Another idea would be to have a single, parametrized test, something like this:

```python
STATS_TO_CHECK = [
    # Hero Class, attribute, expected_value
    (Warrior, "power", 4),
    (Warrior, "armor", 1),
    # ...,
    (Rogue, "power", 3),
    (Rogue, "attack_range", 3),
    # ...
]

@pytest.mark.parametrize("hero_cls,attribute,expected_value", STATS_TO_CHECK)
def test_hero_stats(hero_cls, attribute, expected_value):
    hero = hero_cls(name="Test")
    
    assert hasattr(hero, attribute), \
        f"Hero class {hero_cls.__name__} should have an attribute '{attribute}'"
    actual_value = getattr(hero, attribute)
    assert actual_value == expected_value, \
        f"{hero_cls.__name__}'s {attribute} should be {expected_value}, but was {actual_value}"
```

## Exercise 1 - Complete tests for the ```Position``` class

For the first exercise, you will have to complete the tests inside ```tests/types_test.py```. At the end of the file, you will see that there are two tests to validate the distance between two positions. The idea is that you parametrize this test, using the ```pytest.mark.parametrize``` decorator.

Click the button below for a hint.

This is how you can parametrize a test:

```python
import pytest

MY_PARAMS = [
    # num1, num2, expected_result
    (1, 2, 3),
    (3, 5, 8),
]

@pytest.mark.parametrize("x,y,expected_result", MY_PARAMS)
def test_sum(x, y, expected_result):
    result = x + y
    
    assert result == expected_result, \
        f"Result should have been {expected_result}, but was {result}"
```

## Refactoring ```game_test.py```

If you check the file where the game file is tested, you'll see that although the coverage is at 100%, the file with the tests, ```game.py``` is not that easy to read / maintain, the main reason being that there are quite a lot of tests, so it's not that easy to find your way.

How would you refactor these tests so that they are easier to maintain? If you have a solution, feel free to go ahead. The only requirement: don't remove (and preferrably don't rename) tests.



One option here could be to split the tests into multiple files, so for instance you could create a ```tests/game``` folder, and inside there have files with different tests (e.g. ```do_test.py```, ```available_actions_test.py```, etc... This way of going to a more granular level can make sense in some cases, but it can also be an indication that maybe the ```Game``` class is doing too many things and we should refactor the class itself.

More in general, when something's hard to test or too complex, that usually means that the code is not right, and in the case, the ```Game``` class is probably more on that side.

## Exercise 2 - Bug in the "available actions"

You might have noticed in the previous notebook (or not), but there's a small bug in how the available actions are calculated. Let's show an example:

In [1]:
import numpy as np
np.random.seed(0)

from game_client import GameClient

# Hero classes: Warrior, Rogue, Mage
from questing import Warrior, Rogue, Mage
from questing import Position
hero = Warrior('Albert')
board_width = 3
board_height = 3
num_enemies = 4

game = GameClient(hero=hero, board_width=board_width, board_height=board_height, num_enemies=num_enemies)
# To start, this will just show the board and the available actions
game.play()

[board] Placed Albert - Warrior at position Position(x=0, y=0)
[board] Placed Exit portal at position Position(x=2, y=2)
[board] Placed Archer 1 at position Position(x=1, y=2)
[board] Placed Swordsman 1 at position Position(x=1, y=1)
[board] Placed Apprentice 1 at position Position(x=1, y=0)
[board] Placed Archer 2 at position Position(x=0, y=1)
|      | A(2) | EXIT |
----------------------
| A(1) | S(2) |      |
----------------------
| H    | M(1) |      |
----------------------
Available actions:
[board] Positions with enemies in attack range: [Position(x=0, y=1), Position(x=1, y=0)]
['Move up', 'Move right', 'Attack']


In this case, the available actions include "Move up" and "Move right", but if you try to perform one of these, you'll get an error like this (and no movement  will happen):

```
[board] Can't move to position Position(x=0, y=1), it's not empty!
```

In [2]:
game.play("Move up")

Taking action Move up
[board] Positions with enemies in attack range: [Position(x=0, y=1), Position(x=1, y=0)]
[board] Can't move to position Position(x=0, y=1), it's not empty!
|      | A(2) | EXIT |
----------------------
| A(1) | S(2) |      |
----------------------
| H    | M(1) |      |
----------------------
Available actions:
[board] Positions with enemies in attack range: [Position(x=0, y=1), Position(x=1, y=0)]
['Move up', 'Move right', 'Attack']


You are now responsible for fixing this bug! We'll just guide you through the high-level steps, trying to follow TDD.

### RED - Add a failing test

First thing to do is to add a failing test to the file ```tests/game_test.py```. What do we want to test? The high-level steps would be as follows:

- The context: the "up" position is valid (i.e. ```board.is_valid``` should return True) but not empty (i.e. ```board.is_empty``` should return False.

- The expected result: ```GameActions.MOVE_UP```` shouldn't be one of the available actions.

The test ```test_available_actions_with_move_up``` is a good starting point.

The test could look something like this:
    
```python
def test_available_actions_occupied_slot(game, board):
    up_position = Position(0, 1)

    board.is_valid.return_value = True
    board.is_empty.return_value = False

    available_actions = game.available_actions()

    assert GameActions.MOVE_UP not in available_actions, f"GameActions.MOVE_UP should not be in the available actions"
    board.is_valid.assert_any_call(up_position)
```

### GREEN - Make any changes needed to make the test pass

Next, you will have to make any changes needed in the ```Game.available_actions``` method to make the test pass. HINT: The test is failing because the ```Game.available_actions``` is not taking into account whether the destination position is empty or not, it only checks whether the position is valid.

In the ```available_actions``` method, you will need to change lines like this:

```python
    if self.board.is_valid(up):
        available_actions.append(GameActions.MOVE_UP)

    if self.board.is_valid(down):
        available_actions.append(GameActions.MOVE_DOWN)
```

to this:

```python
    if self.board.is_valid(up) and self.board.is_empty(up):
        available_actions.append(GameActions.MOVE_UP)

    if self.board.is_valid(down) and self.board.is_empty(down):
        available_actions.append(GameActions.MOVE_DOWN)
```

### REFACTOR - Consolidate changes

There's probably no need for a refactor in this case, but feel free to make any refactors you see fit.